In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import zipfile
import os

# Path to your zipped file
zip_path = '/kaggle/input/assignment/FoodSeg103.zip'

# Directory to extract files
extract_dir = '/kaggle/working/'

# Create directory if it doesn't exist
os.makedirs(extract_dir, exist_ok=True)

# Password for the zip file
password = b'LARCdataset9947'  # replace with your actual password

# Unzipping the file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir, pwd=password)


In [2]:
!pip install numpy pandas matplotlib scikit-learn tensorflow keras opencv-python
!pip install opencv-python-headless tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 23.8 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: keras
    Found existing installation: keras 3.3.3
    Uninstalling keras-3.3.3:
      Successfully uninstalled keras-3.3.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.


In [3]:
import os
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Paths (replace with actual paths)
data_dir = '/kaggle/working/FoodSeg103'
image_train_dir = os.path.join(data_dir, 'Images/img_dir/train')
mask_train_dir = os.path.join(data_dir, 'Images/ann_dir/train')
train_list_file = os.path.join(data_dir, 'ImageSets/train.txt')
image_test_dir = os.path.join(data_dir, 'Images/img_dir/test')
mask_test_dir = os.path.join(data_dir, 'Images/ann_dir/test')
test_list_file = os.path.join(data_dir, 'ImageSets/test.txt')

print(image_train_dir)

2024-07-02 16:47:45.764032: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-02 16:47:45.764184: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-02 16:47:45.915525: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


/kaggle/working/FoodSeg103/Images/img_dir/train


In [4]:
import os



# Load image and mask paths
def load_image_mask_paths(file_path, image_dir, mask_dir):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    # Assuming train.txt lists filenames without extensions
    image_paths = [os.path.join(image_dir, line.strip()) for line in lines]  # append .jpg
    mask_paths = [os.path.join(mask_dir, line.strip()).replace('.jpg', '.png') for line in lines]   # append .png

    # Debugging: print paths and check if files exist
    for img_path, mask_path in zip(image_paths, mask_paths):
        if not os.path.exists(img_path):
            print(f"Image file does not exist: {img_path}")
        if not os.path.exists(mask_path):
            print(f"Mask file does not exist: {mask_path}")

    return image_paths, mask_paths

# Load training data
train_image_paths, train_mask_paths = load_image_mask_paths(train_list_file, image_train_dir, mask_train_dir)
test_image_paths, test_mask_paths = load_image_mask_paths(test_list_file, image_test_dir, mask_test_dir)

# Verify loaded paths
print(f"Total images: {len(train_image_paths)}")
print(f"Total masks: {len(train_mask_paths)}")

# Example to load and check an image and its mask
if len(train_image_paths) > 0:
    print(f"Example image path: {train_image_paths[0]}")
    print(f"Example mask path: {train_mask_paths[0]}")
else:
    print("No training data loaded. Check paths and file extensions.")


Total images: 4983
Total masks: 4983
Example image path: /kaggle/working/FoodSeg103/Images/img_dir/train/00000000.jpg
Example mask path: /kaggle/working/FoodSeg103/Images/ann_dir/train/00000000.png


In [5]:
import cv2
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

# Load images and masks
def load_images_and_masks(image_paths, mask_paths, target_size=(256, 256)):
    images = []
    masks = []
    for img_path, mask_path in zip(image_paths, mask_paths):
        img = cv2.imread(img_path)
        if img is None:
            print(f"Warning: Failed to load image at {img_path}")
            continue
        img = cv2.resize(img, target_size)
        img = img / 255.0  # Normalize to [0, 1]
        images.append(img)

        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        if mask is None:
            print(f"Warning: Failed to load mask at {mask_path}")
            continue
        mask = cv2.resize(mask, target_size)
        mask = mask / 255.0  # Normalize to [0, 1]
        mask = np.expand_dims(mask, axis=-1)  # Add channel dimension for grayscale mask
        masks.append(mask)
    return np.array(images), np.array(masks)


# Assuming train_image_paths, train_mask_paths, test_image_paths, and test_mask_paths are defined
train_images, train_masks = load_images_and_masks(train_image_paths, train_mask_paths)
test_images, test_masks = load_images_and_masks(test_image_paths, test_mask_paths)

# Ensure the arrays are not empty
if train_images.size == 0 or train_masks.size == 0:
    raise ValueError("No training data loaded. Please check your dataset paths and file extensions.")

# Data Augmentation parameters
data_gen_args = dict(rotation_range=15,
                     width_shift_range=0.1,
                     height_shift_range=0.1,
                     shear_range=0.01,
                     zoom_range=[0.9, 1.25],
                     horizontal_flip=True,
                     vertical_flip=True,
                     fill_mode='reflect')

# ImageDataGenerator for images and masks
image_datagen = ImageDataGenerator(**data_gen_args)
mask_datagen = ImageDataGenerator(**data_gen_args)

# Fit data generators
seed = 1
image_datagen.fit(train_images, augment=True, seed=seed)
mask_datagen.fit(train_masks, augment=True, seed=seed)

# Create generators
def create_generators(image_datagen, mask_datagen, train_images, train_masks, batch_size=32):
    image_generator = image_datagen.flow(train_images, batch_size=batch_size, seed=seed)
    mask_generator = mask_datagen.flow(train_masks, batch_size=batch_size, seed=seed)
    train_generator = zip(image_generator, mask_generator)
    return train_generator

train_generator = create_generators(image_datagen, mask_datagen, train_images, train_masks)

# Example usage
for images, masks in train_generator:
    print(images.shape, masks.shape)
    break  # Break after the first batch for demonstration


(32, 256, 256, 3) (32, 256, 256, 1)


In [6]:
# !pip uninstall tensorflow
# !pip install tensorflow


In [7]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate

def unet(input_size=(256, 256, 3)):
    inputs = Input(input_size)

    # Encoder
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(512, 3, activation='relu', padding='same')(pool3)
    conv4 = Conv2D(512, 3, activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(pool4)
    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(conv5)

    # Decoder
    up6 = concatenate([UpSampling2D(size=(2, 2))(conv5), conv4], axis=3)
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(up6)
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(conv6)

    up7 = concatenate([UpSampling2D(size=(2, 2))(conv6), conv3], axis=3)
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(up7)
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(conv7)

    up8 = concatenate([UpSampling2D(size=(2, 2))(conv7), conv2], axis=3)
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(up8)
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(conv8)

    up9 = concatenate([UpSampling2D(size=(2, 2))(conv8), conv1], axis=3)
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(up9)
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(conv9)

    outputs = Conv2D(1, 1, activation='sigmoid')(conv9)

    model = Model(inputs=inputs, outputs=outputs)
    return model


In [8]:
train_generator = create_generators(image_datagen, mask_datagen, train_images, train_masks)
validation_generator = create_generators(image_datagen, mask_datagen, test_images, test_masks)

In [ ]:
model = unet()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Assuming train_generator and validation_generator are defined
model.fit(train_generator, validation_data=validation_generator, epochs=10)

Epoch 1/10


I0000 00:00:1719939091.515199     124 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


   5699/Unknown - 8809s 2s/step - loss: 0.3327 - accuracy: 0.4890

In [ ]:
# Assuming create_generators function is defined as before

# Create a validation data generator
validation_generator = create_generators(image_datagen, mask_datagen, test_images, test_masks)

# Example usage
for images, masks in validation_generator:
    print(images.shape, masks.shape)
    break


In [ ]:
model = unet()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(train_generator, epochs=50, batch_size=16, validation_data=validation_generator)

